In [29]:
import gurobipy as gb
import pandas as pd
    
    
#sets:
duties = []             #d element D
bus_trips = []          #t element Tau
car_travels = []        #i element C
car_travel_departures = []           #i element Ć
car_travel_arrivals = []             #j element Ĉ
car_departure_times_from_depot = []  #o element O
car_matches = []        #set of all possible car matches that are active at time o element O


#parameters
duty_cost = []      # c_d
A = {}              # binary matrix A with a_t_d as index
G = {}              # binary matrix G with g_i_d as index
H = {}              # binary matrix H where h_i_j indicates if two car travels can be matched as one round trip
Q = 1               # available staff cars at depot 
M = 5               # Big number can be set to number of seats of staff car
depot_node_id= 0    # Id of depot Node


#Adaptable parameters
beta = 100           # penalty coefficient for uncovered trips


# Loading data

In [30]:
bus_trips_df = pd.read_csv("Instance1\Trips.csv", ";")
bus_trips_df.set_index("TripID", inplace = True)
#Load bus_trips
bus_trips = bus_trips_df.index.values.tolist()
duties_df = pd.read_csv("Instance1\Duties.csv", ";")
duties_df.set_index("DutyID", inplace = True)
#load duties
duties = duties_df.index.values.tolist()
#load duty costs as dictionary
duty_cost = duties_df[["Paid time or cost of duty"]].to_dict()["Paid time or cost of duty"]

#Next Job create A as a dictionary 
car_travels_df = pd.read_csv("Instance1\CarTravels.csv", ";")
#load car travels
car_travels = car_travels_df["CarTravelID"].values.tolist()
staff_car_df = pd.read_csv("Instance1\StaffCar.csv", ";")
#load available staff cars
Q = staff_car_df["Number of staff cars"].values.tolist()[0]
#load depot node id
depot_node_id = staff_car_df["Staff car depot nodeID"].values.tolist()[0]
#load departing car travels (from depot)
idx =car_travels_df["Departure nodeID"]==depot_node_id
car_travel_departures = car_travels_df.loc[idx]["CarTravelID"].values.tolist()
#load arriving car travels (to depot)
car_travel_arrivals = car_travels_df.loc[car_travels_df["Arrival nodeID"]==depot_node_id]["CarTravelID"].values.tolist()
#Load set of all departure times of a staff car from the depot
car_travel_departure_times_from_depot = car_travels_df.loc[idx]["Departure time"].values.tolist()

### creating matricies

In [35]:
A = {}
for key in duties:
    for trip in trips:
        

duties_df.head()


,Paid time or cost of duty,Trips,Car travels
DutyID,,,
1,295,"1, 4,","1, 2, 3,"
2,376,"1, 4, 5,","1, 2, 6,"
3,370,"1, 2, 3, 5,","7, 8, 6,"
4,430,"1, 2, 3, 5, 6,","7, 8, 12,"
5,462,"1, 2, 3, 5, 10,","7, 8, 6, 16,"


# Constructing Model

In [25]:
#construct model
m = gb.Model("DSPSC Model")

#create variables
x_d = m.addVars(duties, vtype=gb.GRB.BINARY)
y_t = m.addVars(bus_trips, vtype=gb.GRB.BINARY)
z_i = m.addVars(car_travels, vtype=gb.GRB.BINARY)
s_i_j = m.addVars(car_travel_departures, car_travel_arrivals, vtype=gb.GRB.BINARY)


#(2)set Objective
obj = (gb.quicksum(duty_cost[duty] * x_d[duty] for duty in duties) 
       + beta*gb.quicksum(y_t[bus_trip] for bus_trip in bus_trips))

m.setObjective(obj,gb.GRB.MINIMIZE)

#set Constraints

#(3)
m.addConstrs(quicksum(A[bus_trip,duty]*x_d[duty] for duty in duties) + y_t[bus_trip] >= 1 for bus_trip in bus_trips)

#(4)
m.addConstrs(quicksum(G[car_travel,duty]*x_d[duty] for duty in duties)
            <= z_i[car_travel]*M for car_travel in car_travels)

#(5)
m.addConstrs(quicksum(G[car_travel,duty]*x_d[duty] for duty in duties)
            >= z_i[car_travel] for car_travel in car_travels)

#(6)
m.addConstrs(quicksum(H[car_travel_depart,car_travel_arriv] *s_i_j[car_travel_depart,car_travel_arriv] 
                      for car_travel_arriv in car_travel_arrivals) ==  z_i[car_travel_depart] 
                      for car_travel_depart in car_travel_departures)

#(7)
m.addConstrs(quicksum(H[car_travel_depart,car_travel_arriv] *s_i_j[car_travel_depart,car_travel_arriv] 
                      for car_travel_depart in car_travel_departures) ==  z_i[car_travel_depart] 
                      for car_travel_arriv in car_travel_arrivals)

#(8)

NameError: name 'quicksum' is not defined